# STAGE C TAG ALONG CODE 

# TO DO 

## Preprocessing
## Feature engineering

In [1]:
# import all necessary packages
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
# load the dataset
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
# display numbers of rows and columns
df.shape

(7043, 21)

In [4]:
# Import necessary methods from scikit-learn
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [6]:
# converting TotalCharges column data type to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors ='coerce')

In [7]:
# Check for null values 
df['TotalCharges'].isna().sum()

11

In [8]:
# Fill all the null values in the TotalCharges column with 0
df['TotalCharges'].fillna(0,inplace= True)

In [9]:
df['TotalCharges'].dtype

dtype('float64')

In [10]:
# check the unique values in the Churn column
df['Churn'].unique()

array(['No', 'Yes'], dtype=object)

In [11]:
# to convert the 'Churn' column to binary values, where 'No' is mapped to 0 and 'Yes' is mapped to 1
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == "Yes" else 0)

In [12]:
df['Churn'].unique()

array([0, 1], dtype=int64)

In [13]:
categorical = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService','OnlineSecurity',
               'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies','Contract', 'PaperlessBilling', 'PaymentMethod']

numerical = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [14]:
# calling StandardScaler() method and fit and transform it on the numerical predictors
scaler = StandardScaler()
numerically_scaled = scaler.fit_transform(df[numerical])
numerical_df = pd.DataFrame(numerically_scaled, columns=numerical)
numerical_df

,tenure,MonthlyCharges,TotalCharges
0,-1.277445,-1.160323,-0.992611
1,0.066327,-0.259629,-0.172165
2,-1.236724,-0.362660,-0.958066
3,0.514251,-0.746535,-0.193672
4,-1.236724,0.197365,-0.938874
...,...,...,...
7038,-0.340876,0.665992,-0.127605
7039,1.613701,1.277533,2.242606
7040,-0.870241,-1.168632,-0.852932
7041,-1.155283,0.320338,-0.870513


In [15]:
# use OneHotEncoder() function to transform the categorical features to numbers.
one_hot_encoded = OneHotEncoder(sparse_output= False)
encoded_category = one_hot_encoded.fit_transform(df[categorical])
encoded_category_df = pd.DataFrame(encoded_category)
encoded_category_df.columns = one_hot_encoded.get_feature_names_out()

In [16]:
encoded_category_df.head()

,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [17]:
# concatenate the two DataFrame 
df_combined = pd.concat([numerical_df, encoded_category_df], axis= 1)

In [18]:
# calling the combined DataFrame
df_combined

,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,-1.277445,-1.160323,-0.992611,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.066327,-0.259629,-0.172165,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,-1.236724,-0.362660,-0.958066,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.514251,-0.746535,-0.193672,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,-1.236724,0.197365,-0.938874,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,-0.340876,0.665992,-0.127605,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
7039,1.613701,1.277533,2.242606,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
7040,-0.870241,-1.168632,-0.852932,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
7041,-1.155283,0.320338,-0.870513,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [19]:
# numbers of rows and columns of the combined DataFrame 
print(f'The combined DataFrame has {df_combined.shape[0]} rows and {df_combined.shape[1]} columns.')

The combined DataFrame has 7043 rows and 46 columns.


In [20]:
# split the dataset into training set and testing set
X = df_combined
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size= 0.2, random_state= 1)

In [21]:
# calling the needed learning algorithms
rfc = RandomForestClassifier(random_state=1)
etc = ExtraTreesClassifier(random_state=1)
xgc = XGBClassifier(random_state=1)
ltb = LGBMClassifier(random_state=1)


## What is the accuracy on the test set using the random forest classifier?

In [22]:
# train a RandomForestClassifier 
rfc.fit(X_train, y_train)
y_pred1 = rfc.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred1)}")
print(f"Precision: {precision_score(y_test, y_pred1)}")
print(f"F1_score: {f1_score(y_test, y_pred1)}")


Accuracy: 0.7906316536550745
Precision: 0.583596214511041
F1_score: 0.556390977443609


In [23]:
# train an ExtraTreesClassifier
etc.fit(X_train, y_train)
y_pred2 = etc.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred2)}")
print(f"Precision: {precision_score(y_test, y_pred2)}")
print(f"F1_score: {f1_score(y_test, y_pred2)}")


Accuracy: 0.7700496806245565
Precision: 0.5384615384615384
F1_score: 0.509090909090909


## What is the accuracy on the test set using the xgboost classifier?

In [24]:
# train a XGBClassifier
xgc.fit(X_train, y_train)
y_pred3 = xgc.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred3)}")
print(f"Precision: {precision_score(y_test, y_pred3)}")
print(f"F1_score: {f1_score(y_test, y_pred3)}")

Accuracy: 0.7934705464868701
Precision: 0.5861027190332326
F1_score: 0.5714285714285715


## What is the accuracy on the test set using the LGBM classifier?

In [25]:
# train a LGBMClassifier
ltb.fit(X_train, y_train)
y_pred4 = ltb.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred4)}")
print(f"Precision: {precision_score(y_test, y_pred4)}")
print(f"F1_score: {f1_score(y_test, y_pred4)}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1521, number of negative: 4113
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000414 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 670
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 46
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.269968 -> initscore=-0.994785
[LightGBM] [Info] Start training from score -0.994785
Accuracy: 0.8133427963094393
Precision: 0.6299694189602446
F1_score: 0.6103703703703703


In [26]:
# setting hyperparameters
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]



In [27]:
hyperparameter_grid = {'n_estimators' : n_estimators, 'min_samples_leaf': min_samples_leaf, 'min_samples_split' : min_samples_split,
                      'max_features' : max_features}
random_search = RandomizedSearchCV(etc, hyperparameter_grid, cv=5, n_iter= 10, scoring= 'accuracy', n_jobs= -1, verbose= 1, random_state=1)

In [28]:
# train the ExtraTreesClassifier with hyperparameters
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=ExtraTreesClassifier(random_state=1),
                   n_jobs=-1,
                   param_distributions={'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 3, 5, 7, 9],
                                        'n_estimators': [50, 100, 300, 500,
                                                         1000]},
                   random_state=1, scoring='accuracy', verbose=1)

## What are the best hyperparameters from the randomized search CV?

In [29]:
# get the best parameter
random_search.best_params_

{'n_estimators': 1000,
 'min_samples_split': 9,
 'min_samples_leaf': 8,
 'max_features': 'sqrt'}

In [30]:
# check the accuracy of the best estimator
accuracy = accuracy_score(y_test,random_search.best_estimator_.predict(X_test))
accuracy   # 0.8041163946061036

0.8041163946061036

## What other hyperparameters can be tuned for ExtraTreeClassifer?

In [31]:
# ANSWER:
# min_weight_fraction_leaf
# max_leaf_nodes

## Find the feature importance using the optimal ExtraTreesClassifier model. Which features are the two most important respectively?

In [32]:
feature_names = list(df_combined.columns)
feature_importances = random_search.best_estimator_.feature_importances_
sorted_features = sorted(zip(feature_importances, feature_names), reverse=True)

# Print the top two most important features
top_two_features = [feature_name for importance, feature_name in sorted_features[:2]]
top_two_features

['Contract_Month-to-month', 'tenure']